# Model Building

### Setup

The `2_Feature_Engineering` notebook takes parameters for which model to build (model), where to store the training data (features_table), and the start (start_date) and end (to_date) dates to use when creating the training data. 

Using these parameters, it creates the training data by calling the `./notebooks/2_Feature_Engineering` with the correct parameters. When the `./notebooks/2_Feature_Engineering` notebook completes, we can start running the other cells to build our model.

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#Used data files
# This is the default feature training data file.
training_table= 'training_data'
# The model we are building is a Random Forest
model_type = 'RandomForest' 

#Databricks parameters to customize the runs
# Input widgets allow you to add parameters to your notebooks and dashboards
dbutils.widgets.removeAll()
dbutils.widgets.text("features_table", training_table)
dbutils.widgets.text("model", model_type)
dbutils.widgets.text("start_date", '2000-01-01')
dbutils.widgets.text("to_date", '2015-10-30')

## Feature Engineering

The `2_Feature_Engineering` notebook run below creates a labeled training data set using the parameters `start_date` and `to_date` to select the time period for training. This data set is stored in the `features_table` specified. After this cell completes, you should see the dataset named `training_data` under the Databricks `Data` icon.

In [0]:
dbutils.notebook.run("2_Feature_Engineering", 600, {"features_table": dbutils.widgets.get("features_table"), 
                                                     "start_date": dbutils.widgets.get("start_date"), 
                                                     "to_date": dbutils.widgets.get("to_date")})

## Model Building and Testing

Load the training data and add databricks paramaters

In [0]:
#Databricks paramaters to customize the runs
dbutils.widgets.text("training_table",training_table)
dbutils.widgets.text("Model", model_type)

In [0]:
spark.catalog.refreshTable(dbutils.widgets.get("training_table")) 
train_data = spark.table(dbutils.widgets.get("training_table"))

# Prepare the Training data

A fundamental practice in machine learning is to calibrate and test your model parameters on data that has not been used to train the model. <br> Evaluation of the model requires splitting the available data into a training portion, a calibration portion and an evaluation portion.<br> Typically, 80% of data is used to train the model and 10% each to calibrate any parameter selection and evaluate your model.

In general random splitting can be used, but since time series data have an inherent correlation between observations; for predictive maintenance problems,<br> a time-dependent spliting strategy is often a better approach to estimate performance. <br> For a time-dependent split, a single point in time is chosen, the model is trained on examples up to that point in time, and validated on the examples after that point. <br> This simulates training on current data and score data collected in the future data after the splitting point is not known. <br> However, care must be taken on labels near the split point. <br> In this case, feature records within 7 days of the split point can not be labeled as a failure, since that is unobserved data.

In [0]:
# define list of input columns for downstream modeling

# We'll use the known label, and key variables.
label_var = ['label_e']
key_cols =['machineID','dt_truncated']

# Then get the remaining feature names from the data
input_features = train_data.columns

# Remove the known label, key variables and a few extra columns we won't need.
remove_names = label_var + key_cols + ['failure','model_encoded','model' ]

# Create the iout features 
input_features = [x for x in input_features if x not in set(remove_names)]

Spark models require a vectorized data frame. We transform the dataset here and then split the data into a training and test set. <br>
We use this split data to train the model on 9 months of data (training data), and evaluate on the remaining 3 months (test data) going forward.

In [0]:
# Import the libraries 
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier

# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
train_data = va.transform(train_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(train_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(train_data)

training = train_data

### Prepare the Testing Data

To evaluate this model, we predict the component failures over the test data set.<br> Since the test set has been created from data the model has not been seen before, it simulates future data. <br> The evaluation can then be generalized to assess how the model could perform when operationalized and used to score new data.

In [0]:
testing_table = 'testing_data'
dbutils.widgets.removeAll()
dbutils.widgets.text("Testing_table",testing_table)
dbutils.widgets.text("Model", model_type)
dbutils.widgets.text("start_date", '2015-11-30')
dbutils.widgets.text("to_date", '2016-02-01')

In [0]:
spark.catalog.setCurrentDatabase("default")
exists = False
for tbl in spark.catalog.listTables():
  if tbl.name == dbutils.widgets.get("Testing_table"):
    exists = True
    break

In [0]:
if not exists:
  dbutils.notebook.run("2_Feature_Engineering", 600, {"features_table": dbutils.widgets.get("Testing_table"), 
                                                       "start_date": dbutils.widgets.get("start_date"), 
                                                       "to_date": dbutils.widgets.get("to_date")})

In [0]:
#Load the data

test_data = spark.table(dbutils.widgets.get("Testing_table"))

# Testing data is prepared using the same steps used for the traning data 

# define list of input columns for downstream modeling

# We'll use the known label, and key variables.
label_var = ['label_e']
key_cols =['machineID','dt_truncated']

# Then get the remaining feature names from the data
input_features = test_data.columns

# Remove the known label, key variables and a few extra columns we won't need.
remove_names = label_var + key_cols + ['failure','model_encoded','model' ]

# Create the iout features 
input_features = [x for x in input_features if x not in set(remove_names)]

# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')

# assemble features
test_data = va.transform(test_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(test_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(test_data)

testing = test_data

## Classification Models

A particular problem in predictive maintenance is machine failures are usually rare occurrences compared to normal operation. This is fortunate for the business as maintenance and saftey issues are few, but causes an imbalance in the label distribution. This imbalance leads to poor performance as algorithms tend to classify majority class examples at the expense of minority class, since the total misclassification error is much improved when majority class is labeled correctly. This causes low recall or precision rates, although accuracy can be high. It becomes a larger problem when the cost of false alarms is very high. To help with this problem, sampling techniques such as oversampling of the minority examples can be used. These methods are not covered in this notebook. Because of this, it is also important to look at evaluation metrics other than accuracy alone.

We will build a Random Forest Classifier:

- **Random Forest Classifier**: A random forest is an ensemble of decision trees. Random forests combine many decision trees in order to reduce the risk of overfitting. Tree ensemble algorithms such as random forests and boosting are among the top performers for classification and regression tasks.

The next code block creates the model. A series of model hyperparametershave also been included to guide your exploration of the model space.

In [0]:
# import the libraries for creating pipelines and model

from pyspark.sql import SparkSession
import numpy as np
from pyspark.ml import PipelineModel
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

 
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                            # Passed to DecisionTreeClassifier
                            maxDepth=15, 
                            maxBins=32, 
                            minInstancesPerNode=1, 
                            minInfoGain=0.0,
                            impurity="gini",
                            # Number of trees to train (>= 1)
                            numTrees=200, 
                            # The number of features to consider for splits at each tree node. 
                            # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                            featureSubsetStrategy="sqrt", 
                            # Fraction of the training data used for learning each  
                            # decision tree, in range (0, 1].' 
                            subsamplingRate = 0.632)
  
# chain indexers and model in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])
  
# train model.  This also runs the indexers.
model = pipeline.fit(training)

###Persist the model

Here we save the model in a paraquet file on DBFS

In other words we have now stored the model on the Azure Databricks files system.

In [0]:
# save model
model.write().overwrite().save("dbfs:/storage/models/" + model_type + ".pqt")

## Model Testing

Load the model so that we can test it on new data

In [0]:
model_pipeline = PipelineModel.load("dbfs:/storage/models/" + dbutils.widgets.get("Model") + ".pqt")

print("Model loaded")
model_pipeline

Model loaded
Out[14]: PipelineModel_6f5b5676a24b

Test the model on new data and calculate a set of model evaluation metrics to help us know how well the model may perform in a production setting.

In [0]:
# make predictions. The Pipeline does all the same operations on the test data
predictions = model.transform(testing)
 
# Create the confusion matrix for the multiclass prediction results
# This result assumes a decision boundary of p = 0.5
conf_table = predictions.stat.crosstab('indexedLabel', 'prediction')
confuse = conf_table.toPandas()
confuse.head()
  
# Log MLflow Metrics and Model
# select (prediction, true label) and compute test error
# select (prediction, true label) and compute test error
# True positives - diagonal failure terms
tp = confuse['1.0'][1]+confuse['2.0'][2]+confuse['3.0'][3]+confuse['4.0'][4]
# False positves - All failure terms - True positives
fp = np.sum(np.sum(confuse[['1.0', '2.0','3.0','4.0']])) - tp
# True negatives 
tn = confuse['0.0'][0]
# False negatives total of non-failure column - TN
fn = np.sum(np.sum(confuse[['0.0']])) - tn
  
# Accuracy is diagonal/total 
acc_n = tn + tp
acc_d = np.sum(np.sum(confuse[['0.0','1.0', '2.0','3.0','4.0']]))
acc = acc_n/acc_d
  
# Calculate precision and recall.
prec = tp/(tp+fp)
rec = tp/(tp+fn)
  
# Calculate F1
FOne = 2.0 * prec * rec/(prec + rec)
  
# Print the evaluation metrics to the notebook
print("Accuracy = %g" % acc)
print("Precision = %g" % prec)
print("Recall = %g" % rec )
print("F1 = %g" % (2.0 * prec * rec/(prec + rec)))
print("")

Accuracy = 0.960505
Precision = 0.985396
Recall = 0.353896
F1 = 0.520764

### Conclusion
This notebook demonstrated how to train, store, load and test a model on new data as well as how to calculate a set of model evaluation metrics to help us assess how well the model may perform in a production setting.

The 4_Scoring_Pipeline notebook takes parameters to define the data to be scored, and using the model created here, calculates the probability of component failure in the machine population specified.